In [1]:
from app.db import Fact, async_session_maker, create_db_and_tables, Base, engine
import textstat

In [4]:
import requests
from datetime import datetime, timedelta
import dotenv
import os
from typing import Optional
from datetime import datetime
from sqlalchemy import select
import json


class NasaFact:
    copyright: Optional[str]
    date: str
    explanation: str
    hdurl: Optional[str]
    media_type: str
    service_version: str
    title: str
    url: str
    level: int

    def __init__(self, copyright: Optional[str], date: str, explanation: str, hdurl: Optional[str], media_type: str, service_version: str, title: str, url: str) -> None:
        self.copyright = copyright
        self.date = date
        self.explanation = explanation
        self.hdurl = hdurl
        self.media_type = media_type
        self.service_version = service_version
        self.title = title
        self.url = url
        self.level = 0


dotenv.load_dotenv()

API_KEY = os.getenv("NASA_API_KEY")

def fetch_apod(start_date, end_date):
    url = f"https://api.nasa.gov/planetary/apod?api_key={API_KEY}&start_date={start_date}&end_date={end_date}"
    response = requests.get(url)
    data = response.json()
    return data

def fetch_apod_today():
    url = f"https://api.nasa.gov/planetary/apod?api_key={API_KEY}"
    response = requests.get(url)
    data = response.json()
    return data

def get_normalized_level(grade: float) -> int:
    min_source = 40.69
    max_source = 80.82

    min_target = 1
    max_target = 5
    level = min_target + ((grade - min_source) * (max_target - min_target) / (max_source - min_source))

    return int(round(level))

async def save_fact_to_db(fact: NasaFact):
    async with async_session_maker() as session:
        session.add(Fact(
            date=fact.date,
            title=fact.title,
            media_type=fact.media_type,
            url=fact.url,
            explanation=fact.explanation,
            level=fact.level
        ))
        await session.commit()


async def fetch_apod_today_save_db():
    data = fetch_apod_today()
    fact = NasaFact(
        data.get("copyright"),
        data.get("date"),
        data.get("explanation"),
        data.get("hdurl"),
        data.get("media_type"),
        data.get("service_version"),
        data.get("title"),
        data.get("url")
    )
    grade = textstat.flesch_reading_ease(fact.explanation)
    level = get_normalized_level(grade)
    fact.level = level
    await save_fact_to_db(fact)

async def fetch_last_month_save_db():
    end_date = datetime.now() - timedelta(days=1)
    start_date = end_date - timedelta(days=30)
    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")

    data = fetch_apod(start_date, end_date)
    facts = []
    for fact in data:
        facts.append(NasaFact(
            fact.get("copyright"),
            fact.get("date"),
            fact.get("explanation"),
            fact.get("hdurl"),
            fact.get("media_type"),
            fact.get("service_version"),
            fact.get("title"),
            fact.get("url"),
        ))
    for fact in facts:
        grade = textstat.flesch_reading_ease(fact.explanation)
        level = get_normalized_level(grade)
        fact.level = level
        await save_fact_to_db(fact)

async def fetch_last_100days_save_db():
    end_date = datetime.now() - timedelta(days=1)
    start_date = end_date - timedelta(days=101)
    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")

    data = fetch_apod(start_date, end_date)
    facts = []
    for fact in data:
        facts.append(NasaFact(
            fact.get("copyright"),
            fact.get("date"),
            fact.get("explanation"),
            fact.get("hdurl"),
            fact.get("media_type"),
            fact.get("service_version"),
            fact.get("title"),
            fact.get("url"),
        ))
    for fact in facts:
        grade = textstat.flesch_reading_ease(fact.explanation)
        level = get_normalized_level(grade)
        fact.level = level
        await save_fact_to_db(fact)

async def fetch_apod_from_json_save_db():
    with open("facts.json", "r", encoding="utf-8") as file:
        data = json.load(file)
        facts = []
        for fact in data:
            facts.append(NasaFact(
                fact.get("copyright"),
                fact.get("date"),
                fact.get("explanation"),
                fact.get("hdurl"),
                fact.get("media_type"),
                fact.get("service_version"),
                fact.get("title"),
                fact.get("url"),
            ))
        for fact in facts:
            grade = textstat.flesch_reading_ease(fact.explanation)
            level = get_normalized_level(grade)
            fact.level = level
            await save_fact_to_db(fact)


In [16]:

r = requests.get("https://apod.nasa.gov/apod/image/2403/The_Dish_Tracking_IM-1_22February2024_04s.jpg", headers=headers)

In [17]:
def download_image(url, path):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
    }
    filename = url.split("/")[-1]
    path = os.path.join(path, filename)
    with open(path, "wb") as file:
        response = requests.get(url, headers=headers)
        file.write(response.content)

In [18]:
with open("facts.json", "r", encoding="utf-8") as file:
    data = json.load(file)
    facts = []
    for fact in data:
        facts.append(NasaFact(
            fact.get("copyright"),
            fact.get("date"),
            fact.get("explanation"),
            fact.get("hdurl"),
            fact.get("media_type"),
            fact.get("service_version"),
            fact.get("title"),
            fact.get("url"),
        ))
        if fact.get("media_type") == "image":
            download_image(fact.get("url"), "imgs")

In [5]:
await fetch_apod_from_json_save_db()

In [58]:
await create_db_and_tables()

In [62]:
async with engine.begin() as conn:
    await conn.run_sync(Base.metadata.drop_all)

In [64]:
end_date = datetime.now() - timedelta(days=1)
start_date = end_date - timedelta(days=101)
start_date = start_date.strftime("%Y-%m-%d")
end_date = end_date.strftime("%Y-%m-%d")

data = fetch_apod(start_date, end_date)
facts = []
for fact in data:
    facts.append(NasaFact(
        fact.get("copyright"),
        fact.get("date"),
        fact.get("explanation"),
        fact.get("hdurl"),
        fact.get("media_type"),
        fact.get("service_version"),
        fact.get("title"),
        fact.get("url"),
    ))
for fact in facts:
    grade = textstat.flesch_reading_ease(fact.explanation)
    level = get_normalized_level(grade)
    fact.level = level

In [65]:
with open("facts.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [66]:
facts = []
for fact in data:
    facts.append(NasaFact(
        fact.get("copyright"),
        fact.get("date"),
        fact.get("explanation"),
        fact.get("hdurl"),
        fact.get("media_type"),
        fact.get("service_version"),
        fact.get("title"),
        fact.get("url"),
    ))
grades = [textstat.flesch_reading_ease(fact.explanation) for fact in facts]

# Minimum and maximum values from the original grades
min_source = min(grades)
max_source = max(grades)

# Min and Max target values for normalization
min_target = 1
max_target = 5

# Applying the min-max normalization formula
normalized_grades = [
    (min_target + ((grade - min_source) * (max_target - min_target) / (max_source - min_source))) for grade in grades
]

levels = [int(round(grade)) for grade in normalized_grades]
levels

[2,
 4,
 3,
 3,
 2,
 4,
 3,
 2,
 2,
 3,
 4,
 2,
 4,
 3,
 4,
 3,
 2,
 3,
 3,
 4,
 3,
 2,
 3,
 3,
 2,
 3,
 3,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 2,
 4,
 3,
 3,
 2,
 2,
 3,
 4,
 2,
 3,
 3,
 3,
 3,
 2,
 3,
 3,
 5,
 2,
 4,
 4,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 4,
 2,
 3,
 3,
 2,
 5,
 3,
 2,
 4,
 3,
 5,
 2,
 3,
 3,
 4,
 2,
 1,
 4,
 3,
 2,
 3,
 4,
 3,
 2,
 3,
 4,
 4,
 3,
 3,
 3,
 3,
 4,
 2,
 2,
 3]

In [70]:


for fact in facts:
    grade = textstat.flesch_reading_ease(fact.explanation)
    level = get_normalized_level(grade)
    async with async_session_maker() as session:
        session.add(Fact(
            date=fact.date,
            title=fact.title,
            media_type=fact.media_type,
            url=fact.url,
            explanation=fact.explanation,
            level=level
        ))
        await session.commit()

In [8]:
# write data to facts.json
with open("facts.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [56]:
with open("facts.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    facts = []
    for fact in data:
        facts.append(NasaFact(
            fact.get("copyright"),
            fact.get("date"),
            fact.get("explanation"),
            fact.get("hdurl"),
            fact.get("media_type"),
            fact.get("service_version"),
            fact.get("title"),
            fact.get("url"),
        ))
    for fact in facts:
        grade = textstat.flesch_reading_ease(fact.explanation)
        level = get_normalized_level(grade)
        fact.level = level
        await save_fact_to_db(fact)


In [5]:
sample_text = "The universe is immense and contains millions of galaxies. Each galaxy contains millions or even billions of stars."

# Example of calculating readability scores
flesch_reading_ease = textstat.flesch_reading_ease(sample_text)
flesch_kincaid_grade = textstat.flesch_kincaid_grade(sample_text)
gunning_fog = textstat.gunning_fog(sample_text)

print(f"Flesch Reading Ease: {flesch_reading_ease}")
print(f"Flesch-Kincaid Grade Level: {flesch_kincaid_grade}")
print(f"Gunning Fog Index: {gunning_fog}")


Flesch Reading Ease: 62.34
Flesch-Kincaid Grade Level: 6.8
Gunning Fog Index: 5.82
